<a href="https://colab.research.google.com/github/babuyashwanth/CNN-models/blob/main/CNNoptimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras-tuner

     |████████████████████████████████| 96 kB 2.8 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=fd50b6a29889552337500ef10c768db9138e36ccdbba55b8b1a3844d5b5a3f4a
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.5.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation='relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 38s]
val_accuracy: 0.9103333353996277

Best val_accuracy So Far: 0.9118333458900452
Total elapsed time: 00h 04m 15s
INFO:tensorflow:Oracle triggered exit


In [13]:
model = tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        27680     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1769568   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,799,178
Trainable params: 1,799,178
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 15s 8ms/step - loss: 0.1275 - accuracy: 0.9523 - val_loss: 0.2712 - val_accuracy: 0.9170
Epoch 5/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0909 - accuracy: 0.9662 - val_loss: 0.3170 - val_accuracy: 0.9080
Epoch 6/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0624 - accuracy: 0.9772 - val_loss: 0.3657 - val_accuracy: 0.9113
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0425 - accuracy: 0.9849 - val_loss: 0.3944 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0357 - accuracy: 0.9873 - val_loss: 0.4640 - val_accuracy: 0.9082
Epoch 9/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0284 - accuracy: 0.9902 - val_loss: 0.4800 - val_accuracy: 0.9157
Epoch 10/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0240 - accuracy: 0.9915 - val_loss: 0.5185 - val_a